<a href="https://colab.research.google.com/github/GabeMaldonado/AIforMedicine/blob/master/AIforMed_W1_Theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes on AI for Medicine -- Week 1

The chest X-ray is one of the most common diagnostic imaging procedures in medicine with **~ 2 billion** chest X-rays taken a year. Chest X-ray interpretation is critical for detecting pneumonia, lung cancer, etc. 

An abonormality in chest X-rays is called a **mass**. A mass is defined as a **lision** or damaged tissue seen on a chest X-ray as greater than 3cm in diameter. 

During training, an algorithm is shown images of chest X-rays labeled with whether they contain a mass or not. The algorithm learns using these images and labels and it produces an output of whether the X-ray contains a mass. 
The algorithm can be a:
*   Deep Learning algorithm
*   Model
*   Convolutional Neural Network

The algorithm produces an output in terms of **scores** which are probabilities that the image contains a mass. 

## Key Challenges for Training Algorithms on Medical Images



1.   Class Imbalance
2.   Multi-Task
3.   Dataset Size


## Class Imbalance

There is not an equal number of examples of non-disease (normal) and disease (mass) in medical datasets. This is a reflection of the **prevalence** or the frequency of disease in the real-world where we see that there are far more examples of normals than of mass, especially when looking at X-rays of a healthy population. In a medical df we might see 100 times as many normal examples as mass examples. 
This creates a problem for the algo which would see mostly normal examples. This yields a model that starts to predict a very low probability of disease for everybody and won't be able to identify an example that has a disease. 





Let's say we have 8 images, 6 of which are normal and 2 are mass.
The value of the **loss function** is 0.3 for each.

Total Loss from Mass Examples   ```= 0.3 * 2 = 0.6```

Total Loss from Normal Examples ```= 0.3 * 6 = 1.8```

Most of the contribution to the loss comes from normal examples rather than from the mass examples, so the algo is optimizing its updates to get the normal examples right and it is not giving much relative weight to the mass examples. This will not produce a very good image classifier. **This is a class imbalance problem.**

The solution to this problem is to modify the loss function to weight the normal and the mass classes differently. We can assign weights to the positive and negative examples. $W_{p}$ for positive and $W_{n}$ for negative. In this case, we want to weight the mass examples more so they can have en equal contribution overall to the loss as the normal examples. Let's select ```6/8``` as the weight we have on the mass examples and ```2/8``` as the weight for the normal examples. 

$W_{p}$ or $W_{n}$ * $loss$

$W_{n} = 2/8 * 0.3 = 0.075 $

$W_{p} = 6/8 * 0.3 = 0.225$

Total Loss from Mass Examples   $= 0.225 * 2 = 0.45$

Total Loss from Normal Examples $= 0.075 * 6 = 0.45$

As we can see, if we sum up the total loss from the mass example we get 0.45 and this is equal to the total loss of the normal examples. 
In general, the weight we would put on the positive class will be the number of negative examples over the total number of examples.

$W_{p} = num negative / num total  = 6/8$ 6 normal examples / 8 (total) examples

$W_{n} = num positive / num total = 2/8$ 2 positive cases / 8 (total) cases

So now the positive and negative class contributions to the loss are the same. This is the idea of modifying the loss using weights and this method is called the **Weighted Loss** to tackle the class imbalance problem. 






### Weighted Loss Equation

- The loss is made up of two terms:
    - $loss_{pos}$: we'll use this to refer to the loss where the actual label is positive (the positive examples).
    - $loss_{neg}$: we'll use this to refer to the loss where the actual label is negative (the negative examples).  
- Note that within the $log()$ function, we'll add a tiny positive value, to avoid an error if taking the log of zero.

$$ loss^{(i)} = loss_{pos}^{(i)} + los_{neg}^{(i)} $$

$$loss_{pos}^{(i)} = -1 \times weight_{pos}^{(i)} \times y^{(i)} \times log(\hat{y}^{(i)} + \epsilon)$$

$$loss_{neg}^{(i)} = -1 \times weight_{neg}^{(i)} \times (1- y^{(i)}) \times log(1 - \hat{y}^{(i)} + \epsilon)$$

$$\epsilon = \text{a tiny positive number}$$

### Re-sampling 

We can also fix the class imbalance problem by re-sampling. The idea here is to re-sample the df to have an equal number of normal and mass examples. 

To achieve this:
*   Group the Normal and Mass examples together
*   From these two groups, sample the images so that there is an equal number of positive and negative samples. This can be done by sampling half of the examples from the mass class and half of the examples from the normal class. Keep in ming that we will not be able to include all of the normal samples when re-sampling and -- we might have more than one copy of the mass examples in the re-sampled dataset. 
If we compute the loss function without the weights (the regular *binary cross-entropy loss*) we will see that there is an equal contribution to the loss from the mass and normal examples. 

There are many variations of this approach, we can under-sample the normal class and over-sample the mass class.

## Multi-task Challenge

Thus far we have been taking about *binary classification* -- identifying the samples as mass or not mass. In the real world we care about classifying the presence or absence of many diseases such as:
*   Mass or No Mass
*   Pneumonia or No Pneumonia
*   Edema or No Edema

One way to attempt to solve these problems would be to have models that would learn each of these tasks. But it is also possible to do all these taks using one model. The advantage being that the model can learn features that are common to identifying more than one disease therefore we can use the data more efficiently. This set-up is caled **Multi-task Learning**. We can train an algorithm to learn all of these taks at the same time. In this learning environment, the examples would contain more information in their labels so instead of having info just about mass-- the label can also contain info about other diseases such as pneumonia and edema. For instance (and for mass, pneumonia, and edema):

**P1 0, 1 ,0**

**P2 0, 0, 1**

P1 would indicate the absence of mass, the presence of pneumonia and the absence of edema.

P2 would indicate the absence of mass, the absence of pneumonia and the presence of edema. 

**Edema** excess fluid in the lungs. 

Also, instead of having one single output as we saw before now we have three different outputs denoting the probability of the three different deseases.

**Prediction Probabilities** 

**P1 0.3, 0.1, 0.8**

**P1 0.1, 0.1, 0.8**

To train such an algorithm, we need to make a modification to the loss function from the binary task to the multitask setting. 

We need to modify the loss function so it can look at the error associated with each disease.

$$ L(X,y_{mass}) + L(X,y_{pneumonia}) + L(X,y_{edema}) $$

We can then represent the new loss function as the sum of the losses over the multiple diseases:

$$ L(X,y)$$
$$ MultiLabel / MultiTask \\ Loss $$ 

**Loss**

**P1 0.52 + 1.00 + 0.70**

**P2 0.05 + 0.05 + 0.10**

Class Imbalance can also be an issue in a multi-taks setting. But just as before, we can also apply the *weighted loss* to a multi-task. 

$$ MultiTask \quad L(X,y_{mass}) + L(X,y_{pneumonia}) + L(X,y_{edema}) $$



$$L(X, y_{class}) =
  \begin{cases}
    - W_{p, \ class}\ log P(Y = 1 | X)       & \quad \text{if } y \text{ = 1}\\
    - W_{n, \ class}\ log P(Y = 0 | X) & \quad \text{if } y \text{ = 0}
  \end{cases}
$$

Here besides having the weights associated with the positive and negative dlabels we also have the associated class. 


## Dataset Size Challenge

For many medical imaging problems the architecture of choice is the Convolutional Neural Network (CovNet or CNN). These are designed to process 2D images such as X-rays. But variants of these are also used for medical signal processing or 3D medical images such CT scans. 
Several CNN architectures such as Inception, ResNet, DenseNet, ResNeXt and EfficientNets are very popular in the image classification field. In the medical field, the standard is to try out multiple models on the task at hand and evaluate which works best. 